In [1]:
import numpy as np
from gym import spaces, Env


In [61]:
action_space = spaces.Box( low=np.array([[22, 8], [22, 8], [0, 1]]),
                high=np.array([[60, 48],[60, 48], [0, 1]]),
                          shape=(3,2),
                dtype=np.int)

In [62]:
action_space.sample()

array([[34, 10],
       [38, 18],
       [ 0,  1]])

In [2]:
action_space = spaces.Box( low=np.array([22, 8, 22, 8, 0]),
                high=np.array([60, 48,60, 48, 1]),
                dtype=np.int)

In [60]:
a = action_space.sample()
a

array([33, 27, 24, 12,  1])

In [5]:
a[0]

29

In [4]:
a = np.array([0, 1, 2])


In [5]:
a*2

array([0, 2, 4])